# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [15]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

In [16]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [18]:
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
#可以看出 not null 的筆數，推出那些欄位有null值
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 10 columns):
Pclass      1309 non-null int64
Name        1309 non-null object
Sex         1309 non-null object
Age         1046 non-null float64
SibSp       1309 non-null int64
Parch       1309 non-null int64
Ticket      1309 non-null object
Fare        1308 non-null float64
Cabin       295 non-null object
Embarked    1307 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 112.5+ KB


In [20]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [21]:
num_features

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [22]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
print('the train_num shape--->',train_num)
df.head()

the train_num shape---> 891


,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [31]:
#可以看出 not null 的筆數，推出那些欄位有null值
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 5 columns):
Pclass    1309 non-null int64
Age       1046 non-null float64
SibSp     1309 non-null int64
Parch     1309 non-null int64
Fare      1308 non-null float64
dtypes: float64(2), int64(3)
memory usage: 61.4 KB


In [32]:
# Age:263 筆 null 
# Fare:1 筆  null 
df.isnull().values.sum()

264

In [33]:
# Age:263 筆 null 
# Fare:1 筆  null 
df.isna().sum()

Pclass      0
Age       263
SibSp       0
Parch       0
Fare        1
dtype: int64

# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [60]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
print('df.isna().sum()------')
print(df.isna().sum())
print()
print()
print('df_m1.isna().sum()------')
print(df_m1.isna().sum())

df.isna().sum()------
Pclass      0
Age       263
SibSp       0
Parch       0
Fare        1
dtype: int64


df_m1.isna().sum()------
Pclass    0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64


In [68]:
print('train_num',train_num)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
#空值補 -1, 做羅吉斯迴歸

#cross_val_score這是驗證用來評分資料準確度的，cv=5是分成5組的意思。
#ref https://ithelp.ithome.com.tw/articles/10197461
#    https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
score = cross_val_score(estimator, train_X, train_Y, cv=5)
print("the score   5 groups  is-->",score)
print()
print("Null  value fill -1: %8.5f"%  score.mean())

train_num 891
the score   5 groups  is--> [0.63687151 0.70391061 0.71910112 0.71348315 0.71751412]

Null  value fill -1:  0.69818


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

In [69]:
# 空值補平均數, 做羅吉斯迴歸
print('df mean==>\n',df.mean())
df_m1 = df.fillna(df.mean())
train_X = df_m1[:train_num]
estimator = LogisticRegression(solver='lbfgs')
print()
print("Null  value fill mean: %8.5f"% cross_val_score(estimator, train_X, train_Y, cv=5).mean())

df mean==>
 Pclass     2.294882
Age       29.881138
SibSp      0.498854
Parch      0.385027
Fare      33.295479
dtype: float64

Null  value fill mean:  0.69594


In [70]:
# 空值補最大值, 做羅吉斯迴歸
print('df max==>\n',df.max())
df_m1 = df.fillna(df.max())
train_X = df_m1[:train_num]
estimator = LogisticRegression(solver='lbfgs')
print()
print("Null  value fill max: %8.5f"%  cross_val_score(estimator, train_X, train_Y, cv=5).mean())

df max==>
 Pclass      3.0000
Age        80.0000
SibSp       8.0000
Parch       9.0000
Fare      512.3292
dtype: float64

Null  value fill max:  0.69378


In [71]:
# 空值補 median, 做羅吉斯迴歸
print('df max==>\n',df.median())
df_m1 = df.fillna(df.median())
train_X = df_m1[:train_num]
estimator = LogisticRegression(solver='lbfgs')
print()
print("Null  value fill median : %8.5f"% cross_val_score(estimator, train_X, train_Y, cv=5).mean())

df max==>
 Pclass     3.0000
Age       28.0000
SibSp      0.0000
Parch      0.0000
Fare      14.4542
dtype: float64

Null  value fill median :  0.69706


In [72]:
# 空值補眾數, 做羅吉斯迴歸
print('df max==>\n',pd.Series(data=df.mode().values[0], index=df.mode().columns))
df_m1 = df.fillna(pd.Series(data=df.mode().values[0], index=df.mode().columns))
train_X = df_m1[:train_num]
estimator = LogisticRegression(solver='lbfgs')
print()
print("Null  value fill mode: %8.5f"% cross_val_score(estimator, train_X, train_Y, cv=5).mean())

df max==>
 Pclass     3.00
Age       24.00
SibSp      0.00
Parch      0.00
Fare       8.05
dtype: float64

Null  value fill mode:  0.70268


1.Null  value fill -1:  0.69818
2.Null  value fill mean:  0.69594
3.Null  value fill max:  0.69378
4.Null  value fill median :  0.69706
5.Null  value fill mode:  0.70268

**嘗試 補-1、平均(mean)、最大值(max)、中位數(median)、眾數(mode)，
  此例以眾數(mode)的效果最佳(mean cross validation scores = 0.70268)

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [75]:
data=df.mode().values[0]
data

array([ 3.  , 24.  ,  0.  ,  0.  ,  8.05])

In [90]:
"""
Your Code Here
"""
#1.原值
df_ml = df.fillna(pd.Series(data=df.mode().values[0], index=df.mode().columns))
train_X = df_m1[:train_num]
estimator = LogisticRegression(solver='lbfgs')
print("Null  value fill mode,不作標準化: %8.5f"% cross_val_score(estimator, train_X, train_Y, cv=5).mean())

Null  value fill mode,不作標準化:  0.70268


In [78]:
df_ml.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [88]:
#2.最小最大化
df_ml = df.fillna(pd.Series(data=df.mode().values[0], index=df.mode().columns))
df_ml = MinMaxScaler().fit_transform(df_ml)
train_X = df_ml[:train_num]

estimator = LogisticRegression(solver='lbfgs')
print("Null  value fill mode,依最小最大化作標準化: %8.5f"% cross_val_score(estimator, train_X, train_Y, cv=5).mean())

Null  value fill mode,依最小最大化作標準化:  0.69711


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [82]:
df_ml

array([[1.        , 0.27345609, 0.125     , 0.        , 0.01415106],
       [0.        , 0.473882  , 0.125     , 0.        , 0.13913574],
       [1.        , 0.32356257, 0.        , 0.        , 0.01546857],
       ...,
       [1.        , 0.48014531, 0.        , 0.        , 0.01415106],
       [1.        , 0.29850933, 0.        , 0.        , 0.01571255],
       [1.        , 0.29850933, 0.125     , 0.11111111, 0.0436405 ]])

In [91]:
#3.標準化
df_ml = df.fillna(pd.Series(data=df.mode().values[0], index=df.mode().columns))
# StandardScaler()：
# Standardize features by removing the mean and scaling to unit variance.
# The standard score of a sample x is calculated as:
# z = (x - u) / s
# where u is the mean of the training samples or zero if with_mean=False, 
# and s is the standard deviation of the training samples or one if with_std=False.
# ref:https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

df_ml = StandardScaler().fit_transform(df_ml)
train_X = df_ml[:train_num]

estimator = LogisticRegression(solver='lbfgs')
print("Null  value fill mode,依StandardScaler作標準化:%8.5f"% cross_val_score(estimator, train_X, train_Y, cv=5).mean())

Null  value fill mode,依StandardScaler作標準化: 0.70268


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [84]:
df_ml

array([[ 0.84191642, -0.51172249,  0.48128777, -0.4449995 , -0.5031762 ],
       [-1.54609786,  0.71038916,  0.48128777, -0.4449995 ,  0.73480864],
       [ 0.84191642, -0.20619458, -0.47908676, -0.4449995 , -0.49012612],
       ...,
       [ 0.84191642,  0.74858015, -0.47908676, -0.4449995 , -0.5031762 ],
       [ 0.84191642, -0.35895854, -0.47908676, -0.4449995 , -0.48770944],
       [ 0.84191642, -0.35895854,  0.48128777,  0.71076309, -0.21108058]])

1.Null  value fill mode,不作標準化:  0.70268
2.Null  value fill mode,依最小最大化作標準化:  0.69711
3.Null  value fill mode,依StandardScaler作標準化: 0.70268

若依null值補mode後，依上述三種方式作比較，不作標準化與依StandardScaler作標準化結果一致。